In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [2]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [3]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-2-851586210124/sagemaker/DEMO-pytorch-mnist


In [4]:
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sagemaker_containers
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)

    parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])

    train(parser.parse_args())


In [5]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.2.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

In [6]:
estimator.fit({'training': inputs})

2020-02-19 21:42:47 Starting - Starting the training job...
2020-02-19 21:42:49 Starting - Launching requested ML instances......
2020-02-19 21:43:48 Starting - Preparing the instances for training......
2020-02-19 21:45:09 Downloading - Downloading input data...
2020-02-19 21:45:39 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-19 21:45:53,945 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-19 21:45:53,948 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-19 21:45:53,959 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-19 21:45:53,646 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-19 21:45


2020-02-19 21:45:52 Training - Training image download completed. Training in progress.Train Epoch: 1 [12800/30000 (43%)] Loss: 1.215920
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.914584
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.937029
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.850368
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.673603
Test set: Average loss: 0.3227, Accuracy: 9100/10000 (91%)

Test set: Average loss: 0.3227, Accuracy: 9100/10000 (91%)

Train Epoch: 2 [6400/30000 (21%)] Loss: 0.546118
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.496909
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.466322
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.378099
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.678142
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.544192
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.493608
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.646309
Test set: Average loss: 0.1935, Accuracy: 9417/10000 (94%)

Test set: Average loss: 0.1935, Accuracy: 9417/10000 (94%)

Train Epoch: 3 [6400/300

In [7]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-----------------!

In [8]:
from IPython.display import HTML
HTML(open("input.html").read())

In [22]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint pytorch-training-2020-02-19-21-42-47-145 of account 851586210124 not found.

In [19]:

estimator.delete_endpoint()